# Cleaning Data: Homework
### Completed by Stephanie Andrews
July 13, 2023

In class, we discussed a few different ways to clean data — simple Python methods, Pandas versions of those methods, and regular expressions (both in plain Python and in Pandas). You are welcome to use whatever combination of those approaches you'd like. If you really want to stretch your skills, try solving the questions a few different ways.

## 0) Load `pets.csv` (provided in CourseWorks)

... and assign the data to a variable named `pets`.

In [1]:
import pandas as pd

In [2]:
pets = pd.read_csv("./pets.csv")
pets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       10 non-null     object
 1   species    10 non-null     object
 2   weight     10 non-null     object
 3   years_old  10 non-null     object
dtypes: object(4)
memory usage: 448.0+ bytes


In [3]:
pets

,name,species,weight,years_old
0,"smith, fido",dog,5 lbs,1
1,"SMITH,, Bella",CAT,7 lbs,10
2,"Ortiz, Milo",dog,15 lbs,5
3,"Smith, Goldie",goldfish,7 oz,???
4,"Kim, Fuzz",cat,3 lbs,5 (five)
5,"Jones, Las Vegas",DOG,12 lbs,unknown
6,Jones; fifi,puppy,2 lbs,0.25
7,"Lee, nemo",gold fish,10 oz,2
8,"Chan, ZIGGY",kitten,2 lbs,0.5
9,"Chan, Zog",dog,20 lbs,7


We're going to clean the data so that you can __programmatically__ answer the following questions:

- Which family owns the most pets? (All pets have been given the surname of the family that owns them. Assume that there's only one family with each surname.)
---
- How many of each kind of animal are there? (Puppies should count as dogs, and kittens as cats.)
---
- What is the total weight of the pets? (And which family's pets weigh the most, total?)
---
- Of the *known* ages, what's the average pet age?

## 1) Create a new column in the DataFrame called `last_name`, containing *just* the pet's surname

... normalized so that `.value_counts()` will count `smith, fido` and `SMITH, Bella` as being from the same family.

In [4]:
import re

In [5]:
pets["last_name"] = pets["name"].apply(lambda x: re.split(r",|;", x)[0].lower())

In [6]:
pets["last_name"].value_counts()

last_name
smith    3
jones    2
chan     2
ortiz    1
kim      1
lee      1
Name: count, dtype: int64

## 2) Check how many distinct last names you see

(There should be 6.)

In [7]:
len(pets["last_name"].unique())

6

## 3) Calculate which family owns the most pets

(All pets have been given the surname of the family that owns them. Assume that there's only one family with each surname.)

In [8]:
counts_df = pets["last_name"].value_counts().to_frame("count").reset_index()

In [9]:
family_pets = counts_df.sort_values(by="count", ascending=False).iloc[0]
print(f"The {family_pets[0].capitalize()} family has {family_pets[1]} pets.")

The Smith family has 3 pets.


## 4) Create a new column called `animal_type`, which standardizes the `species` column

(Puppies should count as dogs, and kittens as cats; and there should only be one spelling of `gold fish`/`goldfish`.)

In [10]:
def standardize_species(x):
    match x.lower().replace(" ", ""):
        case "dog"|"puppy":
            return "dog"
        case "cat"|"kitten":
            return "cat"
        case _:
            return x.lower().replace(" ", "")
    
pets["animal_type"] = pets["species"].apply(lambda x: standardize_species(x))

## 5) How many of each kind of animal are there?

In [11]:
pets["animal_type"].value_counts()

animal_type
dog         5
cat         3
goldfish    2
Name: count, dtype: int64

## 6) Create a new column called `weight_quantity`, which represents the numeric component of the `weight` column, as an integer

(i.e., `"10 oz"` -> `10`)

In [12]:
pets["weight_quantity"] = pets["weight"].apply(lambda x: re.search(r"(\d+\.*\d*)", x).group()).astype(int)

In [13]:
pets

,name,species,weight,years_old,last_name,animal_type,weight_quantity
0,"smith, fido",dog,5 lbs,1,smith,dog,5
1,"SMITH,, Bella",CAT,7 lbs,10,smith,cat,7
2,"Ortiz, Milo",dog,15 lbs,5,ortiz,dog,15
3,"Smith, Goldie",goldfish,7 oz,???,smith,goldfish,7
4,"Kim, Fuzz",cat,3 lbs,5 (five),kim,cat,3
5,"Jones, Las Vegas",DOG,12 lbs,unknown,jones,dog,12
6,Jones; fifi,puppy,2 lbs,0.25,jones,dog,2
7,"Lee, nemo",gold fish,10 oz,2,lee,goldfish,10
8,"Chan, ZIGGY",kitten,2 lbs,0.5,chan,cat,2
9,"Chan, Zog",dog,20 lbs,7,chan,dog,20


## 7) Create a new column called `weight_unit`, which represents the units component of the `weight` column

(i.e., `"10 oz"` -> `"oz"`)

In [14]:
pets["weight_unit"] = pets["weight"].apply(lambda x: re.search(r"([a-z]+)", x).group())
pets

,name,species,weight,years_old,last_name,animal_type,weight_quantity,weight_unit
0,"smith, fido",dog,5 lbs,1,smith,dog,5,lbs
1,"SMITH,, Bella",CAT,7 lbs,10,smith,cat,7,lbs
2,"Ortiz, Milo",dog,15 lbs,5,ortiz,dog,15,lbs
3,"Smith, Goldie",goldfish,7 oz,???,smith,goldfish,7,oz
4,"Kim, Fuzz",cat,3 lbs,5 (five),kim,cat,3,lbs
5,"Jones, Las Vegas",DOG,12 lbs,unknown,jones,dog,12,lbs
6,Jones; fifi,puppy,2 lbs,0.25,jones,dog,2,lbs
7,"Lee, nemo",gold fish,10 oz,2,lee,goldfish,10,oz
8,"Chan, ZIGGY",kitten,2 lbs,0.5,chan,cat,2,lbs
9,"Chan, Zog",dog,20 lbs,7,chan,dog,20,lbs


## 8) Create a new column called `unit_factor`, which should equal `1` when `weight_unit` is `"lbs"` and `0.0625` when `weight_unit` is `"oz"`

(i.e., `"10 oz"` -> `"oz"`)

You could do this by defining a function and using `column.apply(my_function)` or through Pandas' `.replace({ ... })` method, which we didn't cover in class but is somewhat easier here. That would look like this (yes, I'm giving you the answer here):

```python
pets["unit_factor"] = (
    pets["weight_unit"]
    .replace({
        "lbs": 1,
        "oz": 0.0625
    })
)
```

In [15]:
pets["unit_factor"] = (
    pets["weight_unit"]
    .replace({
        "lbs": 1,
        "oz": 0.0625
    })
)

## 8) Create a new column called `weight_lbs`, which calculates the pet's weight in lbs by multiplying `weight_quantity` by `unit_factor`

In [16]:
pets["weight_lbs"] = pets["weight_quantity"]*pets["unit_factor"]

## 9) What is the total weight of the pets?

In [17]:
pets["weight_lbs"].sum()

67.0625

## 10)  And which family's pets weigh the most, total?

In [18]:
pets.groupby("last_name")["weight_lbs"].sum()

last_name
chan     22.0000
jones    14.0000
kim       3.0000
lee       0.6250
ortiz    15.0000
smith    12.4375
Name: weight_lbs, dtype: float64

## 11) Of the known ages, what's the average pet age?

(You should get `3.84375`; if not, make sure you're accounting for numbers like `0.5` correctly.)

In [19]:
def standardize_age(x):
    return float(re.search(r"(\d+\.*\d*)", x).group())

pets["years_num"] = pets["years_old"].apply(lambda x: standardize_age(x) if re.search(r"(\d+\.*\d*)", x) else None)

In [20]:
pets["years_num"].dropna().mean()

3.84375

---

---

---